In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

import argparse
from copy import deepcopy
import logging
from matplotlib.cm import get_cmap, ScalarMappable
from matplotlib.colors import Normalize
import matplotlib.pyplot as plt
import pandas as pd
import scipy
from sklearn.metrics import r2_score
import sys

sys.path.append('..')

from data_utils import *

sys.path.append('/om2/user/eisenaj/code/DeLASE')
from delase import *
from stability_estimation import *
from parameter_choosing import *
from performance_metrics import *
from plot import *

# Arguments + Parameters

In [3]:
data_processing_args_dir = "/scratch2/weka/millerlab/eisenaj/ChaoticConsciousness/data_processing_args"

# Batch Queue

In [4]:
mp_args = dict(
    # ----------------------
    # MULTIPROCESSING ARGUMENTS
    # ----------------------
    USE_TORCH=True,
    USE_CUDA=False,
    DTYPE='torch.DoubleTensor',
    NUM_WORKERS=20,
#     NUM_WORKERS=4,
    USE_LOGGING=True,
    LOG_DIR="/om2/user/eisenaj/code/shell_scripts/DeLASE",
    LOG_NAME='mp_delase',
    LOG_LEVEL=logging.DEBUG,
    RESULTS_DIR='/scratch2/weka/millerlab/eisenaj/ChaoticConsciousness/performance_results',
    
    COMPUTE_IP=True,
    COMPUTE_CHROOTS=False,
    SAVE_JACOBIANS=False,
    TRACK_RESEEDS=True,
    NORM = False,
    NORM_AIC=True,
    
    QUEUE_FULL_SESSION=True,

    # ----------------------
    # DYNAMICAL SYSTEMS ARGUMENTS
    # ----------------------
    parameter_grid=ParameterGrid(
                        window_vals=np.array([15000]),
                        matrix_size_vals=np.array([750]),
                        r_vals=np.array([750]),
                        reseed=True,
                        reseed_vals=np.array([1, 5, 10, 15, 20, 30, 40, 50, 100, 150, 200, 250, 300, 400, 500, 750, 1000])
                    ),
    T_pred=15000, # time steps
    dt = 0.001, # s
    integrated_performance_kwargs = dict(
        metrics=['autocorrel_correl', 'fft_correl', 'fft_r2'], 
        weights='equal',
        num_lags=500,
        max_freq=200,
        fft_n=1000,
    ),
#     integrated_performance_kwargs = {},
    stability_max_freq = 500,
    stability_max_unstable_freq = 125,
    
)
mp_args = argparse.Namespace(**mp_args)

In [5]:
# session_list = [
#     'MrJones-Anesthesia-20160113-01',
#     'Mary-Anesthesia-20160809-01',
#     'MrJones-Anesthesia-20160105-01',
#     'Mary-Anesthesia-20160818-02',
#     'MrJones-Anesthesia-20160121-01',
#     'Mary-Anesthesia-20160831-02',
#     'MrJones-Anesthesia-20160123-01'
# ]

# session_list = [
#     'Mary-Anesthesia-20160902-02',
#     'Mary-Anesthesia-20160908-02',
#     'Mary-Anesthesia-20160914-02',
#     'Mary-Anesthesia-20160916-02',

#     'MrJones-Anesthesia-20160125-01',
#     'MrJones-Anesthesia-20160201-01',
#     'MrJones-Anesthesia-20160206-01',
#     'MrJones-Anesthesia-20160210-01',
#     'MrJones-Anesthesia-20160301-01'
# ]

session_list = os.listdir("/scratch2/weka/millerlab/eisenaj/ChaoticConsciousness/grid_search_results")

In [6]:
session_list = session_list[1:2]
# session_list = session_list[4:]

In [7]:
session_list

['MrJones-Anesthesia-20160301-01']

In [8]:
norm_folder = "NOT_NORMED" if not mp_args.NORM else "NORMED"
for session in tqdm(session_list):
    print("-"*30)
    print(f'SESSION = {session}')
    print("-"*30)
    # -----------------------------------------
    # LOAD DATA AND DIRECTORY
    # -----------------------------------------
    print("loading data...")
    all_data_dir = '/scratch2/weka/millerlab/eisenaj/datasets/anesthesia/mat'
    data_class = get_data_class(session, all_data_dir)

    os.environ["HDF5_USE_FILE_LOCKING"] = "FALSE"
    # variables = ['electrodeInfo', 'lfp', 'lfpSchema', 'sessionInfo', 'trialInfo', 'unitInfo']
    # session_vars, T, N, dt = load_session_data(session, all_data_dir, variables, data_class=data_class, verbose=False)
    # electrode_info, lfp, lfp_schema, session_info, trial_info, unit_info = session_vars['electrodeInfo'], session_vars['lfp'], session_vars['lfpSchema'], session_vars['sessionInfo'], session_vars['trialInfo'], session_vars['unitInfo']

    variables = ['electrodeInfo', 'lfpSchema', 'sessionInfo', 'trialInfo', 'unitInfo']
    session_vars, T, N, dt = load_session_data(session, all_data_dir, variables, data_class=data_class, verbose=False)
    electrode_info, lfp_schema, session_info, trial_info, unit_info = session_vars['electrodeInfo'], session_vars['lfpSchema'], session_vars['sessionInfo'], session_vars['trialInfo'], session_vars['unitInfo']

    eyes_close = session_info['eyesClose'][-1] if isinstance(session_info['eyesClose'], np.ndarray) else session_info['eyesClose'] 
    section_times = dict( 
            pre=(0, session_info['drugStart'][0]),
            induction=(session_info['drugStart'][0], eyes_close),
            during=(eyes_close, session_info['drugEnd'][1]),
            post=(session_info['drugEnd'][1], T*dt)
    )
    sections = list(section_times.keys())
#     areas = np.unique(electrode_info['area'])
    # areas = np.hstack([areas, 'all'])
    areas = ['all']
    
    print("data loaded!")
    print("loading directory")
    
    chunked_folder = [f for f in os.listdir(os.path.join(all_data_dir, data_class)) if f.startswith(session) and 'chunked' in f]
    if len(chunked_folder) == 0:
        print(f"Chunked folder for {session} not found in {os.path.join(all_data_dir, data_class)}")
    else:
        chunked_folder = chunked_folder[0]

    directory = pd.read_pickle(os.path.join(all_data_dir, data_class, chunked_folder, 'directory'))
    
    print("directory loaded!")
    
    # -----------------------------------------
    # PREP AND QUEUE
    # -----------------------------------------
    
    session_results = {}
    session_results[session] = {}
    # data_processing_df_grid = deepcopy(mp_args.data_processing_df)

    grid_results_dir = '/scratch2/weka/millerlab/eisenaj/ChaoticConsciousness/grid_search_results'
    areas = os.listdir(os.path.join(grid_results_dir, session, norm_folder))

    for area in areas:
        df = pd.DataFrame({'window': [], 'matrix_size': [], 'r': [], 'AICs': [], 'time_vals': [], 'file_paths': []}).set_index(['window', 'matrix_size', 'r'])
        for f in os.listdir(os.path.join(grid_results_dir, session, norm_folder, area)):
            t = float(f.split('_')[0])
            file_path = os.path.join(grid_results_dir, session, norm_folder, area, f)
            df_new = pd.DataFrame(pd.read_pickle(file_path))
            if np.isnan(df_new.AIC).sum() > 0:
                print(file_path)
            df_new = df_new.set_index(['window', 'matrix_size', 'r'])
            for i, row in df_new.iterrows():
                if i in df.index:
                    df.loc[i, 'AICs'].append(row.AIC)
                    df.loc[i, 'time_vals'].append(t)
                    df.loc[i, 'file_paths'].append(file_path)
                else:
                    df.loc[i] = {'AICs': [row.AIC], 'time_vals': [t], 'file_paths': [file_path]}

        df = df.loc[df.index.sortlevel()[0]]
        session_results[session][area] = df
    
    print("Grid search results loaded!")

    # ================
    # INDIVIDUAL AREAS
    # ================ 
    window, matrix_size, r, all_results = combine_grid_results({key: result for key, result in session_results[session].items() if key !='all'})

    # ================
    # ALL AREAS
    # ================ 
    if 'all' in session_results[session].keys():
        window_all, matrix_size_all, r_all, all_results_all = combine_grid_results({key: result for key, result in session_results[session].items() if key =='all'})

    # ================
    # QUEUE A JOB FOR EACH AREA
    # ================ 
    for area in areas:
        data_processing_rows = []
        if area != 'all':
            stride = window
            mp_args.parameter_grid = ParameterGrid(
                window_vals = np.array([window]),
                matrix_size_vals = np.array([matrix_size]),
                r_vals = np.array([r]),
                reseed=mp_args.parameter_grid.reseed,
                reseed_vals=mp_args.parameter_grid.reseed_vals,
            )
        else:
            stride = window_all
            mp_args.parameter_grid = ParameterGrid(
                window_vals = np.array([window_all]),
                matrix_size_vals = np.array([matrix_size_all]),
                r_vals = np.array([r_all]),
                reseed=mp_args.parameter_grid.reseed,
                reseed_vals=mp_args.parameter_grid.reseed_vals,
            )
        results_dir = os.path.join(mp_args.RESULTS_DIR, os.path.join(session, 'NORMED' if mp_args.NORM else 'NOT_NORMED', area))

        if area == 'all':
            unit_indices = np.arange(len(electrode_info['area']))
        else:
            unit_indices = np.where(electrode_info['area'] == area)[0]

        t = 0
        while t + window <= len(lfp_schema['index'][0]):
            finished = True
            file_path = os.path.join(results_dir, f"{int(t)/1000}_window_{window}_{mp_args.parameter_grid.expansion_type}_{matrix_size}")
            if not os.path.exists(file_path):
                row = dict(
                    session=session,
                    area=area,
                    window_start=t*mp_args.dt,
                    window_end=(t + window + mp_args.T_pred)*mp_args.dt,
                    directory=directory,
                    dimension_inds=unit_indices
                )
                data_processing_rows.append(row)
            t += stride
        data_processing_df = pd.DataFrame(data_processing_rows)
        if len(data_processing_df) > 0:
            mp_args.data_processing_df = data_processing_df
            data_processing_path = os.path.join(data_processing_args_dir, f"mp_args_{session}_FULL{'_NORMED' if mp_args.NORM else ''}_{area}.pkl")
            mp_args.QUEUE_FULL_SESSION = False
            if area == 'all':
                mp_args.SAVE_JACOBIANS = True
            else:
                mp_args.SAVE_JACOBIANS = False
            pd.to_pickle(vars(mp_args), data_processing_path)

            if not mp_args.USE_CUDA:
                os.system(f"sbatch --gres=gpu:0 --ntasks=1 --cpus-per-task={int(mp_args.NUM_WORKERS) + 4} --mem={int(mp_args.NUM_WORKERS*8)}GB /om2/user/eisenaj/code/shell_scripts/DeLASE/mp_delase.sh {data_processing_path}")
            else:
                os.system(f"sbatch --gres=gpu:{mp_args.NUM_WORKERS} /om2/user/eisenaj/code/shell_scripts/DeLASE/mp_delase.sh {data_processing_path}")

  0%|          | 0/1 [00:00<?, ?it/s]

------------------------------
SESSION = MrJones-Anesthesia-20160301-01
------------------------------
loading data...
data loaded!
loading directory
directory loaded!
Grid search results loaded!
Submitted batch job 30489920
Submitted batch job 30489921
Submitted batch job 30489922
Submitted batch job 30489923
Submitted batch job 30489924


In [101]:
os.system("squeue -u eisenaj")

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
          30391956 millerlab cha-jupy  eisenaj  R   22:54:03      1 node103
          30411908    normal   delase  eisenaj  R       1:48      1 node102
          30411909    normal   delase  eisenaj  R       1:48      1 dgx002
          30411910    normal   delase  eisenaj  R       1:48      1 node104
          30411911    normal   delase  eisenaj  R       1:48      1 node078
          30411907    normal   delase  eisenaj  R       1:49      1 node106


0

In [86]:
os.system('scancel {30411891..30411895}')

0

In [93]:
del ParameterGrid